# Usage

Run setup cell and cell below "Chapter 4"-heading to interactively simulate different schemes for the linear advection equation as well as the SPS. 

# Setup

In [1]:
from __future__ import print_function
%matplotlib inline

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Chapter 4

## The linear advection equation

Note that the option limiter is only valid for the second-order upwind scheme (SOU) with limiter. 


In [2]:
%run src/advection_test_suite.py 

interactive_plot = interactive(interactive_advection_test, 
t=widgets.FloatSlider(min=0, max=3, step=0.1, value=1.5), 
resolution=widgets.IntSlider(min=16, max=256, step=4, value=128), 
time_order= widgets.IntSlider(min=1, max=4, step=1, value=1),
scheme = ["centered difference", "upwind", "second-order upwind", "sou with limiter", "lax-wendroff"],
test = ["gaussian", "tophat"],
limiter = [
    "DONORCELL"  , 
    "BEAMWARMING",
    "LAXWENDROFF",
    "MINMOD"     ,
    "VANALBADA"  ,   
    "SUPERBEE"   ,
    "MC"         ,
    "VANLEER"
],
take_snapshot = False
)
output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=1.5, description='t', max=3.0), IntSlider(value=128, description='reso…

## The phase scheme

In [2]:
%run src/sps_test_suite.py

def pick_scheme(scheme_name, test_name, resolution, eta):
    c = getBaseConfig()
    test, testConfig, potential = test_list[nice_test_list[test_name]]
    scheme, schemeConfig = scheme_list[nice_scheme_list[scheme_name]]
    schemeConfig(c)
    testConfig(c)

    c["resolution"] = int(c["resolution"] * resolution)
    c["dpi"] = 200
    c["figsize"] = [3.54, 2.54]
    c["plotDebug"] = False
    c["hbar"] = eta 
    c["m"] = 1

    solver = scheme(c, test)
    solver.setExternalPotentialFunction(potential)
    return scheme_name, test_name, solver 


scheme_picker = interactive(pick_scheme, 
scheme_name = nice_scheme_list.keys(),
test_name = nice_test_list.keys(),
resolution = widgets.FloatSlider(min=.1, max=4, step=.1, value=1),
eta = widgets.FloatSlider(min=.25, max=4, step=.05, value=1)
)
scheme_picker

interactive(children=(Dropdown(description='scheme_name', options=('phase scheme', 'phase scheme without quant…

In [4]:

def run(t, take_snapshot, back_in_time, reset, useAdaptiveTimestep, dt):
    sname, tname, solver = scheme_picker.result
    if reset:
        scheme_picker.result = pick_scheme(sname, tname, 1, solver.eta)
        sname, tname, solver = scheme_picker.result

    solver.tEnd = solver.config["tEnd"] * t/100
    solver.config["savePlots"] = take_snapshot
    solver.useAdaptiveTimestep = useAdaptiveTimestep
    solver.dt = dt

    if take_snapshot:
        solver.config["dpi"] = 600
    else:
        solver.config["dpi"] = 100

    solver.run(enableBackward=back_in_time)
    animation.drawFrame(solver)
    plt.show()


interactive_plot= interactive(run,
t=widgets.FloatSlider(min=0, max=100, step=1, value=0), 
take_snapshot = False,
back_in_time = False,
reset = False,
useAdaptiveTimestep = True,
dt = widgets.FloatLogSlider(min=-5, max=-2, step=1e-5, value=1e-4)
)

output = interactive_plot.children[-1]
output.layout.height = '500px'
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='t', step=1.0), Checkbox(value=False, description='ta…